In [ ]:
 saved outside of the current session

In [ ]:
#importing required libraries
import csv
import numpy as np
import pandas as pd
import os
from shutil import copyfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense , Flatten,Dropout
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from tensorflow.keras import Sequential
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# mapping image and its class
image_class_dic={}
with open("../input/state-farm-distracted-driver-detection/driver_imgs_list.csv") as f:
    reader=csv.reader(f)
    next(reader)
    for row in reader:
        image_class_dic[row[2]]=row[1]

In [ ]:
#training directory path
TRAIN_DIR="../input/state-farm-distracted-driver-detection/imgs/train"
#creating dict to convert class to its meaning
class_name = dict()
class_name["c0"] = "SAFE_DRIVING"
class_name["c1"] = "TEXTING_RIGHT"
class_name["c2"] = "TALKING_PHONE_RIGHT"
class_name["c3"] = "TEXTING_LEFT"
class_name["c4"] = "TALKING_PHONE_LEFT"
class_name["c5"] = "OPERATING_RADIO"
class_name["c6"] = "DRINKING"
class_name["c7"] = "REACHING_BEHIND"
class_name["c8"] = "HAIR_AND_MAKEUP"
class_name["c9"] = "TALKING_TO_PASSENGER"

In [ ]:
#creating model and its architecture and giving lossfunction and optimizer

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(100,100,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
#creating image data generator


train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(100, 100),
    batch_size=120,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, # same directory as training data
    target_size=(100, 100),
    batch_size=120,
    class_mode='categorical',
    subset='validation') # set as validation data


In [ ]:
#if our accuracty is not improving in further steps we are going to stop it early
cs=EarlyStopping(monitor="val_acc",patience=2,min_delta=0.01)

In [ ]:
plot_model(model,show_shapes=True,show_layer_names=True)

In [ ]:
#fitting our model using fit_generator 
#our model history is stored in model_history
#so that we can see how our model accuracy and loss changed over the concurrent epochs
model_history=model.fit_generator(train_generator,epochs=25,verbose=1,validation_data=validation_generator,callbacks=[cs])

In [ ]:
sns.set()
plt.figure(figsize=(10,7))
plt.plot(model_history.history["accuracy"],label="MODEL_ACCURACY")
plt.plot(model_history.history["val_accuracy"],label="VALIdATION_ACCURACY")
plt.xlabel("No. of Epochs",size=20)
plt.ylabel("Accuracy",size=20)
plt.title("ACCURACY vs VALIDATION_ACCURACY",size=20,color='b')
plt.legend()
plt.show()

In [ ]:
sns.set()
plt.figure(figsize=(10,7))
plt.plot(model_history.history["loss"],label="LOSS")
plt.plot(model_history.history["val_loss"],label="VALIDATION_LOSS")
plt.xlabel("No. of Epochs",size=20)
plt.ylabel("Loss",size=20)
plt.title("LOSS vs VALIDATIAON_LOSS",size=20,color='b')
plt.legend()
plt.show()

In [ ]:
#THERE IS NO NEED FOR US TO DO SEPARATE TRAINING AND TESTING SINCE WE SPLITTED VALIDATION DATA WHICH IS 
#COMPLETELY DIFFERENT FROM TRAINING DATA
#or else we can use our validation generator as test 